In [3]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter
import copy

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from segtok.segmenter import split_single

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
pd.options.display.max_colwidth = 50
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Creating base

In [4]:
new_test=pd.read_csv('..//..//bases/new_test_variants.csv')
new_test_texts = pd.read_csv('..//..//bases/new_test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
new_test_final=pd.merge(new_test,new_test_texts,how="left",on="ID")

In [6]:
leaks=pd.read_csv('..//..//bases/s1_add_train.csv')
leaks_1=pd.DataFrame([leaks["ID"],leaks.drop("ID",axis=1).idxmax(axis=1).map(lambda x: x.lstrip('class'))])
leaks_2=leaks_1.T
leaks_2.columns=["ID","Class"]

In [8]:
train = pd.read_csv('..//..//bases/training_variants')
test = pd.read_csv('..//..//bases/test_variants')

In [9]:
train_texts = pd.read_csv('..//..//bases/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
test_texts = pd.read_csv('..//..//bases/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")

In [10]:
train = pd.merge(train, train_texts, how='left', on='ID')
test = pd.merge(test, test_texts, how='left', on='ID')

In [11]:
leaks_3=pd.merge(leaks_2,test[test.ID.isin(leaks_2.ID)])
leaks_final=pd.merge(leaks_3,test_texts[test_texts.ID.isin(leaks_3.ID)])

In [12]:
new_train = pd.concat([train,leaks_final]) #adding first stage

In [18]:
merge_match = new_test.merge(new_train, left_on=['Gene', 'Variation'], right_on = ['Gene', 'Variation'])
Index_leak = merge_match.ID_x - 1
new_test_index = [item for item in new_test_final.index if item not in list(Index_leak)]
test_no_leaks = new_test_final.iloc[new_test_index]
test_no_leaks

,ID,Gene,Variation,Text
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...
5,6,CHEK2,E239K,The nuclei that laboratories solution p53 KIT ...
6,7,CHST3,T141M,Myeloid differentiation 88 (MyD88) is the key ...
7,8,RNF6,G244D,Human ESCCs 2 occur frequently worldwide (1) ....
8,9,SPAST,C448Y,large were of activity growth this product tol...
10,11,SCN4A,V445M,Endometrial carcinoma is the most common gynec...
14,15,ERBB2,G746S,The protein-kinase family is the most frequent...
15,16,TP53,Y234S,Among the best-studied therapeutic targets in ...
16,17,RAB27A,A87P,"Introduction In recent years, a better unders..."


In [19]:
#### process the train and test set together
data_all = pd.concat((new_train, new_test_final), axis=0, ignore_index=True)
data_all = data_all[['Class', 'Gene', 'ID', 'Variation', 'Text']] # just reordering
data_all_backup = data_all[:] ## We keep backup in case we need to use again
data_all

,Class,Gene,ID,Variation,Text
0,1,FAM58A,0,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,2,CBL,1,W802*,Abstract Background Non-small cell lung canc...
2,2,CBL,2,Q249E,Abstract Background Non-small cell lung canc...
3,3,CBL,3,N454D,Recent evidence has demonstrated that acquired...
4,4,CBL,4,L399V,Oncogenic mutations in the monomeric Casitas B...
5,4,CBL,5,V391I,Oncogenic mutations in the monomeric Casitas B...
6,5,CBL,6,V430M,Oncogenic mutations in the monomeric Casitas B...
7,1,CBL,7,Deletion,CBL is a negative regulator of activated recep...
8,4,CBL,8,Y371H,Abstract Juvenile myelomonocytic leukemia (JM...
9,4,CBL,9,C384R,Abstract Juvenile myelomonocytic leukemia (JM...


In [20]:
#Transform Amino Acid (AA) Letter to their three-letter abbreviation in order to find them in the text when they appear
One_to_Three_AA = {'C': 'Cys', 'D': 'Asp', 'S': 'Ser', 'Q': 'Gln', 'K': 'Lys',
         'I': 'Ile', 'P': 'Pro', 'T': 'Thr', 'F': 'Phe', 'N': 'Asn', 
         'G': 'Gly', 'H': 'His', 'L': 'Leu', 'R': 'Arg', 'W': 'Trp', 
         'A': 'Ala', 'V': 'Val', 'E': 'Glu', 'Y': 'Tyr', 'M': 'Met'}
pattern = re.compile('|'.join(One_to_Three_AA.keys()))

# Adding extra feature(s)

In [21]:
# Feature for the length of the text
data_all["Text_words"] = data_all["Text"].map(lambda x: len(str(x).split(" ")))

# Substitutions (subs)
## Functions for pre-processing of subs

In [22]:
# find_sub return the substituions that are in text and those that are not
def find_sub(data):    
    Boolean = [data.Variation[i][:-1] in data.Text[i] or #case 1.
               pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1]) # case2
               in data.Text[i]  for i in data.index] ## because new indexing we use 
    
    sub_in_text = data[Boolean]
    not_Boolean = [not i for i in Boolean]  
    sub_not_in_text = data[not_Boolean]
    
    return sub_in_text, sub_not_in_text

# find_sub_numberChange searches for other number of a substitution i.e. G12V -> G_V because sometimes mistake in entry
# Is currently without One_to_three substitution or Variation[:-1] only the full variation
def find_sub_numberChange(data):
    Booleans = [] #will contain the different Booleans if found in text
    for i in data.index:
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino
        Boolean = bool(re.search(new_regex, data.Text[i]))
        Booleans.append(Boolean)
        
    sub_number_in_text = data[Booleans]
    not_Boolean = [not i for i in Booleans]  
    sub_number_no_text = data[not_Boolean]
    
    return sub_number_in_text, sub_number_no_text

# for substitutions that are still not found, use other keywords
def find_sub_pattern(data, pattern):    
    Boolean = [pattern in data.Text[i] for i in data.index] ## because new indexing we use 
    
    sub_in_text = data[Boolean]
    not_Boolean = [not i for i in Boolean]  
    sub_not_in_text = data[not_Boolean]
    
    return sub_in_text, sub_not_in_text


In [23]:
##### get_sentences_sub use a window to extract sentences where the subs appear. 
# If window_left & window_right = 0 => just taking the sentences with subs

def get_sentences_sub(data, splitted_sentences):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        one_to_three_variation = pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1])
        Variation = data.Variation[i][:-1]    
        for j in range(len(sentences)):                              
            if (Variation in sentences[j]) or (one_to_three_variation in sentences[j]):
                new_regex = re.escape(Variation) + r"[\S]*" ###  r"[\S]*" because we look for Variation[:-1] not just Variation
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) #case 1
                ### We add the space to ' placeholderMutation' because sometimes there are letters in front of it
                new_regex = re.escape(one_to_three_variation) + r"[\S]*"
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) #case 2
                sentences_with_sub[i].extend(sentences[j:j+1])
                
    return sentences_with_sub

##### get_sentences_sub_number use a window to extract sentences where the subs appear that have different number i.e. G12V -> G_V

def get_sentences_sub_number(data, splitted_sentences, window_left, window_right):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub_number = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino
        
        for j in range(len(sentences)):
            Boolean = bool(re.search(new_regex, sentences[j]))            
            if Boolean:
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) # Again replacing the sentence with placeholder
                sentences_with_sub_number[i].extend(sentences[j:j+1])
    
    return sentences_with_sub_number

# for substitutions that are still not found, use other keywords
def get_sentences_pattern(data, splitted_sentences, pattern, window_left, window_right):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        for j in range(len(sentences)):                              
            if (pattern in sentences[j]):
                sentences_with_sub[i].extend(sentences[j:j+1])               
    
    return sentences_with_sub

In [24]:
###### We use a window when sentences are too low. For 5 <= LENGTH <= 10: window of 1, for <= 5: window of 2
###### 

def get_window_sub(data, splitted_sentences, lengths):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        length = lengths[i]
        if length < 2:
            window=3
        elif length >= 6:
            window=1
        else:
            window=2
        
        one_to_three_variation = pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1])
        Variation = data.Variation[i][:-1] 
        all_sentences = []
        for j in range(len(sentences)):
            if (Variation in sentences[j]) or (one_to_three_variation in sentences[j]):
                new_regex = re.escape(Variation) + r"[\S]*" ###  r"[\S]*" because we look for Variation[:-1] not just Variation
                other_regex = re.escape(one_to_three_variation) + r"[\S]*"
                for sentence in sentences[max(j-window,0) : min(j+1+window, len(sentences)-1)]:
                    sentence = re.sub(new_regex, ' placeholderMutation', sentence) #case 1
                    sentence = re.sub(other_regex, ' placeholderMutation', sentence) #case 2 after case 1
                    all_sentences.append(sentence)
        sentences_with_sub[i] = all_sentences
                    
    return sentences_with_sub

def get_window_sub_number(data, splitted_sentences, lengths):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub_number = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        length = lengths[i]
        if length < 2:
            window=3
        elif length >= 6:
            window=1
        else:
            window=2
            
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino    
        
        all_sentences = []
        for j in range(len(sentences)):
            Boolean = bool(re.search(new_regex, sentences[j]))            
            if Boolean:
                for sentence in sentences[max(j-window,0) : min(j+1+window, len(sentences)-1)]:
                    sentence = re.sub(new_regex, ' placeholderMutation', sentence)
                    all_sentences.append(sentence)
        sentences_with_sub_number[i] = all_sentences
    
    return sentences_with_sub_number



In [25]:
#### Converts list of sentences into one string of sentences for each document => to use for tfidf etc.
def sentences_to_string(sentences_list):
    sentence_strings = []
    for sentences in sentences_list:
        sentence_string =  ' '.join(str(sentence) for sentence in sentences)
        sentence_strings.append(sentence_string)
    
    return sentence_strings 

## Subs processing of the data set 

In [26]:
######### First find those that have the format of being a substitution in data
data_all['Substitutions_var'] = data_all.Variation.apply(lambda x: bool(re.search('^[A-Z]\\d+[A-Z*]$', x))*1) #multiplying by 1 converts True to 1, False to 0 => Maybe modify this later?
data_all['Stop_codon_var'] = data_all.Variation.apply(lambda x: bool(re.search('[*]', x))*1) #multiplying by 1 converts True to 1, False to 0
data_sub = data_all[data_all['Substitutions_var']==1] ### Now we know the index of where a substitution occurs - the data_sub
print("Length of total subs: %i" %len(data_sub)) # other ways to process it like finding the word 'mutation'

Length of total subs: 3820


### Subs inside the text

#### Subs inside the text: all sentences

In [27]:
## First consider the subs that appear in text
sub_text, sub_no_text = find_sub(data_sub) 

In [28]:
## use tokenizer to split into sentences of all the subs in text 
NLTK_sub = [sent_tokenize(sub_text.Text[i]) for i in sub_text.index] 

In [29]:
NLTK_window = copy.deepcopy(NLTK_sub) # a deep copy is necessary besauce something magical and strange happening without it
NLTK_copy = copy.deepcopy(NLTK_sub) # again for backup purposes

In [30]:
# extract window for the sub sentences where they appear
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_sentences = get_sentences_sub(sub_text[:], NLTK_copy[:]) # choosing for window 0 as default now
sub_sentences_string = sentences_to_string(sub_sentences)

In [31]:
# Replace text in data_all
data_all.Text.iloc[sub_text.index] = sub_sentences_string #iloc for indexing based on integers
data_all.Text.iloc[sub_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1       Using select c-CBL somatic mutations such as S...
2       Using select c-CBL somatic mutations such as S...
3       Most of the changes were novel, although 4 cas...
4       Finally, the third group constituted mutations...
5       Finally, the third group constituted mutations...
6       The second group of mutants (M374V,  placehold...
8       We investigated the mechanism by which CBL- pl...
9       We investigated the mechanism by which CBL-Y37...
10      Finally, the third group constituted mutations...
11      Purified PCR amplicons (CBLWT, CBLC381A, CBL p...
12      Purified PCR amplicons (CBLWT, CBLC381A, CBLK3...
13      Purified PCR amplicons (CBLWT, CBL placeholder...
14      The second group of mutants (M374V, V430M,  pl...
15      Purified PCR amplicons (CBLWT, CBLC381A, CBLK3...
17      When introduced into Lin- Sca1+ c-Kit+ (LSK) H...
18      The second group of mutants ( placeholderMutat...
19      When introduced into Lin- Sca1+ c-Kit+ (LSK) H...
20      Using 

#### Subs inside the text: sentences with length <=10

In [32]:
######### We rerun for window. Length<2 : w = 3, Length >=6: w = 1, Length in between: w = 2
indexes = [index for index, sentences in enumerate(sub_sentences) if len(sentences) <= 10]
sentence_lengths = [len(sentences) for index, sentences in enumerate(sub_sentences) if len(sentences) <= 10]
NLTK_sub_window = [NLTK_window[i] for i in indexes]

In [33]:
new_index = sub_text.index[indexes]
sub_window = sub_text.loc[new_index] # gets the subs with the low length
sub_window #1663 cases 

,Class,Gene,ID,Variation,Text,Text_words,Substitutions_var,Stop_codon_var
3,3,CBL,3,N454D,Recent evidence has demonstrated that acquired...,5625,1,0
4,4,CBL,4,L399V,Oncogenic mutations in the monomeric Casitas B...,6248,1,0
5,4,CBL,5,V391I,Oncogenic mutations in the monomeric Casitas B...,6248,1,0
6,5,CBL,6,V430M,Oncogenic mutations in the monomeric Casitas B...,6248,1,0
10,4,CBL,10,P395A,Oncogenic mutations in the monomeric Casitas B...,6248,1,0
14,5,CBL,14,P428L,Oncogenic mutations in the monomeric Casitas B...,6248,1,0
17,4,CBL,17,Q367P,Acquired uniparental disomy (aUPD) is a common...,3013,1,0
18,5,CBL,18,M374V,Oncogenic mutations in the monomeric Casitas B...,6248,1,0
19,4,CBL,19,Y371S,Acquired uniparental disomy (aUPD) is a common...,3013,1,0
21,4,CBL,21,C396R,Oncogenic mutations in the monomeric Casitas B...,6248,1,0


In [34]:
sub_window_sentences = get_window_sub(sub_window[:], NLTK_sub_window[:], sentence_lengths) 
sub_window_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_window_sentences] # removes duplicates
sub_window_string = sentences_to_string(sub_window_sentences)

In [35]:
# Finally: replacing those texts in data_all
data_all.Text.iloc[new_index] = sub_window_string
data_all.Text.iloc[new_index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


3       HRM analysis of CBL exons 8 and 9 in blast cri...
4       Only V430M mutant was on the borderline with t...
5       Only V430M mutant was on the borderline with t...
6       Figure 4 Figure 4 Structures of wild type (gre...
10      Only V430M mutant was on the borderline with t...
14      Finally, the Y371H mutation not only abolished...
17      Transformed NIH3T3 cells showed PI3 kinase-dep...
18      Figure 4 Figure 4 Structures of wild type (gre...
19      Transformed NIH3T3 cells showed PI3 kinase-dep...
21      HEK293T cells were transfected with plasmids e...
22      Only V430M mutant was on the borderline with t...
23      Overall survival (A) and progression-free surv...
24      We fully sequenced the coding exons of these g...
25      Overall survival (A) and progression-free surv...
27      Both families contain compound heterozygotes. ...
29      Here we describe 2 families in which 2 TERT mu...
30      Both families contain compound heterozygotes. ...
32      Both f

### Subs with a different number inside the text

#### Subs with a different number inside the text: all sentences


In [36]:
# now the subs that don't appear in text: one reason is different number in the substitution. f.e. G12V -> G13V 
sub_number_text, sub_number_no_text = find_sub_numberChange(sub_no_text) # 131 cases with number change

In [37]:
## use tokenizer to split into sentences of subs that have different number in text 
NLTK_sub_number = [sent_tokenize(sub_number_text.Text[i]) for i in sub_number_text.index]
NLTK_sub_number_window = copy.deepcopy(NLTK_sub_number) # a deep copy is necessary besauce something magical and strange happening without it
NLTK_sub_number_copy = copy.deepcopy(NLTK_sub_number) # again for backup purposes

In [38]:
# extract window for the sub sentences where they appear
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_number_sentences = get_sentences_sub_number(sub_number_text[:], NLTK_sub_number_copy[:], window_left = 0, window_right = 0)
sub_number_string = sentences_to_string(sub_number_sentences)

In [39]:
data_all.Text.iloc[sub_number_text.index] = sub_number_string #iloc for indexing based on integers
data_all.Text.iloc[sub_number_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


44      His-tagged versions of the catalytic region of...
49      His-tagged versions of the catalytic region of...
61      His-tagged versions of the catalytic region of...
63      His-tagged versions of the catalytic region of...
66      His-tagged versions of the catalytic region of...
73      While both were missense mutations, in silico ...
208      placeholderMutation and T847I were also found...
213     Complex mutation  placeholderMutation and dele...
267     As a positive control, we introduced the well-...
275     And in the series of Pallis and colleagues (12...
306     Recurrent mutations in H3F3A, which encodes th...
307     Histone H3-K42me2 may regulate transcription  ...
308     Recurrent mutations in H3F3A, which encodes th...
521     Both combinations contained mutations of codon...
607     In the present study, we have focused on two d...
632     Such mutant amino acid specificity was also ap...
730     In contrast, ectopic expression of FGFR4  plac...
1081    Recent

#### Subs with a different number inside the text: sentences with length <= 10

In [40]:
######### ######### We rerun for window. Length<2 : w = 3, Length >=6: w = 1, Length in between: w = 2
indexes = [index for index, sentences in enumerate(sub_number_sentences) if len(sentences) <= 10]
sentence_lengths = [len(sentences) for index, sentences in enumerate(sub_number_sentences) if len(sentences) <= 10]
NLTK_window = [NLTK_sub_number[i] for i in indexes]

In [41]:
new_index = sub_number_text.index[indexes]
sub_number_window = sub_number_text.loc[new_index] # gets the subs with the low length
sub_number_window #89 cases 

,Class,Gene,ID,Variation,Text,Text_words,Substitutions_var,Stop_codon_var
44,4,PTPRT,44,T1365M,"Tyrosine phosphorylation, regulated by protein...",1958,1,0
49,4,PTPRT,49,R1343L,"Tyrosine phosphorylation, regulated by protein...",1958,1,0
61,4,PTPRT,61,R1209W,"Tyrosine phosphorylation, regulated by protein...",1958,1,0
208,2,EGFR,208,V774A,Purpose: Clinical features of epidermal growth...,4691,1,0
213,2,EGFR,213,V774M,Purpose: Clinical features of epidermal growth...,4691,1,0
267,7,EGFR,267,G810S,Purpose: Epidermal growth factor receptor (EGF...,4201,1,0
275,2,EGFR,275,L838P,Purpose: Clinical features of epidermal growth...,4691,1,0
308,8,H3F3A,308,G35V,The DNA entry and exit points on the nucleosom...,17067,1,0
521,4,TP53,521,E286K,The transcription factor and tumor suppressor ...,4177,1,0
632,1,FBXW7,632,G423R,Background Melanoma is a heterogeneous tumor ...,10992,1,0


In [42]:
sub_window_sentences = get_window_sub_number(sub_number_window[:], NLTK_window[:], sentence_lengths) 
sub_window_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_window_sentences] # removes duplicates
sub_window_string = sentences_to_string(sub_window_sentences)
sub_window_sentences[0]

['1) would result in polypeptides devoid of C-terminal phosphatase catalytic domains, thereby leading to inactivation of the phosphatase.',
 'To evaluate whether tumor-specific point mutations alter phosphatase activity, we biochemically tested mutant versions of PTPRT, the most frequently mutated PTP in the superfamily.',
 'Mutations in both intracellular PTP domains (D1 and D2) were evaluated.',
 'His-tagged versions of the catalytic region of wild-type PTPRT, two D1 mutants (Q987K and N1128I), and three D2 mutants (R1212W, R1346L, and  placeholderMutation) were produced in bacteria and analyzed for phosphatase activity by using 6,8-difluoro-4-methylumbelliferyl phosphate (DiFMUP) as a substrate (Fig.',
 '2) (12).',
 'All D1 and D2 mutants had reduced phosphatase activity compared with the wild-type protein (Fig.',
 '2).']

In [43]:
data_all.Text.iloc[new_index] = sub_window_string
data_all.Text.iloc[new_index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


44      1) would result in polypeptides devoid of C-te...
49      1) would result in polypeptides devoid of C-te...
61      1) would result in polypeptides devoid of C-te...
208     Identifications of very rare EGFR mutations ma...
213     By searching the database (38), we found that ...
267     To test more directly whether the mutations EG...
275     In the large series of Shigematsu and colleagu...
308     Somatic mutations in the H3.3-ATRX-DAXX chroma...
521     Table 1. Intragenic suppressor mutations obtai...
632     Only colorectal tumors had KRAS A146 mutations...
730     None of the somatic alleles of EPHA3, ERBB4, F...
1083    EZH2 is the catalytic subunit of the PRC2 comp...
1085    EZH2 is the catalytic subunit of the PRC2 comp...
1091    Fifteen of 16 mutations in ERCC2, a nucleotide...
1092    Fifteen of 16 mutations in ERCC2, a nucleotide...
1139    Summary During the past decade, the treatment ...
1144    The H1112L and H1112Y mutations were not detec...
1149    Mutati

#### Subs that are still not found

In [44]:
# subs that are still not found, look based on a pattern like the word 'mutat' in text
sub_pattern_text, sub_pattern_no_text = find_sub_pattern(sub_number_no_text, 'mutat') # 

In [45]:
sub_pattern_no_text # only 14 texts that still don't include sentences with the word 'mutat', of which three nulls and low text_words length

,Class,Gene,ID,Variation,Text,Text_words,Substitutions_var,Stop_codon_var
140,5,EGFR,140,I491M,The accurate determination of perfluoroalkyl s...,430,1,0
145,2,EGFR,145,K467T,The accurate determination of perfluoroalkyl s...,430,1,0
259,2,EGFR,259,S464L,The accurate determination of perfluoroalkyl s...,430,1,0
416,4,TP53,416,P151S,The effects of chlorpromazine on various prope...,718,1,0
1109,1,FANCA,1109,S1088F,null,1,1,0
1407,6,FGFR3,1407,K508M,null,1,1,0
1613,4,VHL,1613,L158Q,The case of a 40-year-old woman with severe ed...,53,1,0
1936,7,CARD11,1936,G116S,Left ventricular (LV) remodeling is a signific...,3681,1,0
1942,7,CARD11,1942,E127G,Left ventricular (LV) remodeling is a signific...,3681,1,0
2302,7,JAK1,2302,R724H,Regulatory T (T reg) cells have a major role i...,8287,1,0


In [46]:
## use tokenizer to split into sentences of subs that have keyword 'mutat'
NLTK_sub_pattern = [sent_tokenize(sub_pattern_text.Text[i]) for i in sub_pattern_text.index]

In [47]:
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_pattern_sentences = get_sentences_pattern(sub_pattern_text[:], NLTK_sub_pattern[:], 'mutat', window_left = 0, window_right = 0)
sub_pattern_string = sentences_to_string(sub_pattern_sentences)

In [48]:
data_all.Text.iloc[sub_pattern_text.index] = sub_pattern_string #iloc for indexing based on integers
data_all.Text.iloc[sub_pattern_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


46      Mutations in CBL, encoding an E3 ubiquitin lig...
80      This D1 domain contains only two serine residu...
102     Identification of a high-risk disease-causing ...
104     In this study, we profiled 70 colorectal cance...
106     Identification of a high-risk disease-causing ...
108     Most Lynch Syndrome-associated CRCs are report...
111     Identification of a high-risk disease-causing ...
112     Germ-line MSH6 mutations, which are rare in HN...
136     In 2011, whole-exome sequencing studies showed...
151     We investigated the association between mutati...
155     Clinical data and epidermal growth factor rece...
161     Clinical data and epidermal growth factor rece...
168     Currently available methods of EGFR mutation d...
178     Those were analyzed for EGFR and k-ras (Kirste...
182     Purpose: Clinical reports about responsiveness...
185     ‘Classical' mutations in the EGFR tyrosine kin...
215     Purpose: Clinical features of epidermal growth...
222     Findin

# Non-substitutions (no subs)
## Functions for pre-processing of no_subs like fusions, deletions, truncations, etc.

In [49]:
## Gets different non-subs variation types by using regex
def variation_regex(data, pattern): 
    Boolean = [bool(re.search(pattern, i, re.IGNORECASE)) for i in data.Variation]
    data_regex = data[Boolean] 
    not_Boolean = [not i for i in Boolean]  
    data_no_regex = data[not_Boolean]
    
    return (data_regex, data_no_regex)

In [50]:
def custom_find(data):
    l=["mutat","variat","fusion","deleti","amplific","framesh",
    "truncat","fs","exon","dupl"]
    Boolean={}
    for word in l:
        Boolean[word] = [re.search(word,data.Text[i]) for i in data.index] ## because new indexing we use 
    Bool=[any(tup) for tup in zip(Boolean["mutat"],Boolean["variat"],Boolean["fusion"],
        Boolean["deleti"],Boolean["amplific"],Boolean["truncat"],
        Boolean["fs"],Boolean["framesh"],Boolean["exon"],Boolean["dupl"])]
    no_sub_in_text=data[Bool]
    no_Bool=[not i for i in Bool]
    no_sub_not_in_text=data[no_Bool]
    return no_sub_in_text,no_sub_not_in_text


In [51]:
def get_sentences_pattern(data, splitted_sentences, pattern, window_left, window_right):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        for j in range(len(sentences)):                              
            if (pattern in sentences[j]):
                sentences_with_sub[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])               
    
    return sentences_with_sub

In [52]:
def get_sentences_nosub(data, splitted_sentences, window_left, window_right):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub = [[] for _ in range(len(data))]

    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        l=["mutat","variat","fusion","deleti","amplific","framesh",
        "truncat","fs","exon","dupl"]     
        for j in range(len(sentences)):    
            for keyword in l:
                if keyword in sentences[j]:
                    sentences_with_sub[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])
    return sentences_with_sub

## No subs processing of the data set 

In [53]:
#### First find those that have the format of being a non-substitutions in the data

# Initialize some of the variables already (because we splice them, could be filled with NA's otherwise)
# Represents 783 from train and test set
data_all['gene_fusion_var'] = 0
data_all['Deletion_var'] = 0
data_all['del_or_ins_var'] = 0

####### Fusions : 'Fusions' ############
data_all['Fusion_var'] = data_all.Variation.apply(lambda x: bool(re.search('^fusion', x, re.IGNORECASE))*1) #multiplying by 1 converts True to 1, False to 0
_ , new_data_all = variation_regex(data_all, '^fusion')  #37 cases

###### Fusions: 'Gene-Gene fusion' ########
data_all['gene_fusion_var'].loc[new_data_all.index] = new_data_all.Variation.apply(lambda x: bool(re.search('fusion', x, re.IGNORECASE))*1)
_ , new_data_all = variation_regex(new_data_all, 'fusion') #160 cases

####### Deletions: 'Deletions' ############
data_all['Deletion_var'].loc[new_data_all.index] = new_data_all.Variation.apply(lambda x: bool(re.search('^del', x, re.IGNORECASE))*1)
_, new_data_all = variation_regex(new_data_all, '^del') # 88 cases

####### Deletions & Insertions wheteher together or seperately (doesn't make a big difference IMO)
data_all['del_or_ins_var'].loc[new_data_all.index] = new_data_all.Variation.apply(lambda x: bool(re.search('del|ins', x, re.IGNORECASE))*1)
# 196 cases

###### Amplifications #########
data_all['Amplification_var'] = data_all.Variation.apply(lambda x: bool(re.search('ampl', x, re.IGNORECASE))*1) # 79 cases

###### Truncations ########### Don't forget there are 'Truncating mutations'  '_trunc' 
data_all['Truncation_var'] = data_all.Variation.apply(lambda x: bool(re.search('trunc', x, re.IGNORECASE))*1) # 118 cases

####### Exons #########
data_all['exon_var'] = data_all.Variation.apply(lambda x: bool(re.search('exon', x, re.IGNORECASE))*1) 

####### Frameshift mutations ########
data_all['frameshift_var'] = data_all.Variation.apply(lambda x: bool(re.search('fs', x, re.IGNORECASE))*1) # 22 cases

####### Duplications ##############
data_all['dup_var'] = data_all.Variation.apply(lambda x: bool(re.search('dup', x, re.IGNORECASE))*1)

####### Hypermethylation ##########
data_all['overexpression_var'] = data_all.Variation.apply(lambda x: bool(re.search('Overexpression', x, re.IGNORECASE))*1) 

####### Splice ##########
data_all['splice_var'] = data_all.Variation.apply(lambda x: bool(re.search('splice', x, re.IGNORECASE))*1) 

####### Hypermethylation ##########
data_all['hypermethylation_var'] = data_all.Variation.apply(lambda x: bool(re.search('hypermethylation', x, re.IGNORECASE))*1)

####### DNA binding ##########
data_all['DNA_binding_var'] = data_all.Variation.apply(lambda x: bool(re.search('DNA', x, re.IGNORECASE))*1)

####### Null ##########
data_all['null_var'] = data_all.Variation.apply(lambda x: bool(re.search('null', x, re.IGNORECASE))*1)


C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [41]:
# Remember there are still unassigned variations that we can assign to for the next run (21 in total)
all_variations = data_all.loc[:, "Substitutions_var":]
data_all[(all_variations.T == 0).all()]

,Class,Gene,ID,Variation,Text,Text_words,Substitutions_var,Stop_codon_var,gene_fusion_var,Deletion_var,...,Amplification_var,Truncation_var,exon_var,frameshift_var,dup_var,overexpression_var,splice_var,hypermethylation_var,DNA_binding_var,null_var
31,7.0,TERT,31,Promoter Mutations,Sequencing studies have identified many recurr...,21096,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150,7.0,EGFR,150,EGFRvIII,Alterations ofthe EGFR gene occur frequently i...,25595,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152,7.0,EGFR,152,EGFRvII,Abstract Glioblastomas with EGFR amplificatio...,12517,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
170,7.0,EGFR,170,EGFRvV,Abstract The epidermal growth factor recepto...,9121,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211,7.0,EGFR,211,EGFR-KDD,Oncogenic EGFR mutations are found in 10-35% o...,2724,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238,7.0,EGFR,238,EGFRvIV,Tumor cells often subvert normal regulatory me...,7536,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350,1.0,CDH1,350,Epigenetic Silencing,"In diffuse gastric carcinoma, despite common E...",3955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,4.0,EP300,354,R1627,The transcriptional coactivator p300/CBP (CREB...,3275,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
355,4.0,EP300,355,C1385,The transcriptional coactivator p300/CBP (CREB...,3275,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
631,4.0,FBXW7,631,Copy Number Loss,This study focused on a cell cycle regulatory ...,29536,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# Preprocess all the nosubs together
nosub=data_all[data_all['Substitutions_var']==0]

In [43]:
nosub_text, nosub_no_text = custom_find(nosub)
nosub_no_text # only 4 not found, of which three nulls

,Class,Gene,ID,Variation,Text,Text_words,Substitutions_var,Stop_codon_var,gene_fusion_var,Deletion_var,...,Amplification_var,Truncation_var,exon_var,frameshift_var,dup_var,overexpression_var,splice_var,hypermethylation_var,DNA_binding_var,null_var
1277,1.0,ARID5B,1277,Truncating Mutations,null,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1639,6.0,FLT1,1639,Amplification,null,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1911,1.0,FOXP1,1911,Truncating Mutations,The transcriptional network of the androgen re...,863,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3426,2.0,AURKB,3426,Amplification,null,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [44]:
# find the sentences containing the nosubs
NLTK_nosub = [sent_tokenize(nosub_text.Text[i]) for i in nosub_text.index]

In [45]:
nosub_sentences = get_sentences_nosub(nosub_text[:], NLTK_nosub[:], window_left = 0, window_right = 0) # Retrieves sentences where keyword is included
nosub_sentences_string = sentences_to_string(nosub_sentences)

In [46]:
### Replacement inside the Text of data_all
data_all.Text.iloc[nosub_text.index] = nosub_sentences_string #iloc for indexing based on integers
data_all.Text.iloc[nosub_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0       Cyclin M, an orphan cyclin, is the product of ...
7       In this study, we determined the frequency of ...
16      Germline and somatic mutations that deregulate...
31      Sequencing studies have identified many recurr...
33      Studies with comparative genomic hybridization...
41      Here, we report that deletion of the miRNA pro...
43      Here, we report that deletion of the miRNA pro...
48      Experimental Design  Genome-wide DNA methylati...
70      Both, human SHQ1 and NAP57 (as an N-terminal f...
71      Polyclonal anti-cyclin D2 and anti-cyclin D3 a...
72      Here we report the association of three Mre11 ...
74      The Mre11 complex specifies ssDNA endonuclease...
75      Polyclonal anti-cyclin D2 and anti-cyclin D3 a...
86      A Sac I (-1 195)-BamHI (+ 79) fragmnent and a ...
87      A Sac I (-1 195)-BamHI (+ 79) fragmnent and a ...
88      To determine whether RYBP binding to MDM2 is n...
96      Mutations in the known high-risk genes, BRCA1,...
100     These 

In [189]:
#Not used
################ The dummy variables for Gene and Text ##################
## TODO: also use dummy for Text? There are 135 shared Genes and 142 shared Text between train and Leaks!  len(set(train.Text) & set(Leaks.Text))
#data_all_dummy = data_all_backup[['Gene', 'Text']] # drop those columns we don't need as dummy.
#X_dummy = pd.get_dummies(data_all_dummy) # converts categorical variables into dummy variable. From len set => 269 genes + 2090 texts
#X_dummy_train = X_dummy[:train.shape[0]]
#X_dummy_test = X_dummy[train.shape[0]:]
#dummy_names = X_dummy.columns.values #### To remember names if you want to check again what Gene or Text used
#X_dummy = X_dummy.values

# Adding gene dummy

In [54]:
new_train = data_all.iloc[:len(train_all)]
new_test = data_all.iloc[len(train_all):]

In [55]:
def add_missing_dummy_columns( d, columns ):
    missing_cols = set( columns ) - set( d.columns )
    for c in missing_cols:
        d[c] = 0
def fix_test_columns( d, columns ):  

    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols:
        print ("extra columns:", extra_cols)

    d = d[ columns ]
    return d

In [56]:
svd = TruncatedSVD(n_components=25, n_iter=12, random_state=26)

one_hot_gene = pd.get_dummies(new_train['Gene'])
one_hot_gene_test=pd.get_dummies(new_test["Gene"])
one_hot_gene_test=fix_test_columns(one_hot_gene_test,one_hot_gene.columns)
truncated_one_hot_gene = svd.fit_transform(one_hot_gene.values)
truncated_one_hot_gene_for_test=svd.transform(one_hot_gene_test.values)

extra columns: {'YARS', 'RGS9', 'WISP3', 'TTK', 'NDUFS3', 'CRB1', 'SLC19A2', 'PER2', 'STK19', 'EPHA2', 'ABCB11', 'AKAP9', 'SLC7A7', 'ROCK1', 'EDAR', 'SLC22A4', 'SLC17A5', 'BAG3', 'RPS26', 'GCM2', 'SLC7A9', 'MCC', 'GRM6', 'CILP', 'RAD54B', 'FLNB', 'PRKRA', 'ZFPM2', 'TRPM1', 'MYOT', 'EPHA5', 'COX15', 'KRIT1', 'LRP6', 'TRPC6', 'DYNC2H1', 'MOCS1', 'SCO1', 'KERA', 'SLC22A5', 'HABP2', 'WNT4', 'AGXT', 'SYT6', 'CRNKL1', 'DCC', 'SLC25A12', 'CLDN16', 'LITAF', 'SCN4A', 'SLC33A1', 'NEK8', 'POLH', 'TINF2', 'RBBP8', 'LCT', 'XRCC1', 'ATP2C1', 'ASS1', 'RP1', 'IKBKAP', 'ADAMTS13', 'SLC25A15', 'PNPO', 'RAB27A', 'CRLF1', 'DPM1', 'TSHR', 'EPHB2', 'ABCC6', 'RPS19', 'GLE1', 'CST3', 'GALK1', 'APOL1', 'PDE8B', 'OTOF', 'PTCH2', 'DNAH5', 'SEPT9', 'STK33', 'DNAI1', 'SF3B2', 'CYP7B1', 'BMPR1B', 'SIX3', 'KISS1R', 'AURKC', 'SCN9A', 'PLA2G6', 'ALOX12B', 'KCNE2', 'SLC25A13', 'AP3B1', 'BFSP2', 'KLF11', 'TNFRSF11A', 'RECQL4', 'MPDU1', 'GYG1', 'CHST3', 'PHOX2B', 'MOCS2', 'GPHN', 'LRP5', 'B4GALT7', 'GJB3', 'CSF1R', 'TGM5

In [57]:
genes_train=pd.DataFrame(truncated_one_hot_gene,columns=["tsvd_gene"+ str(x) for x in range(0,25)])
genes_test=pd.DataFrame(truncated_one_hot_gene_for_test,columns=["tsvd_gene"+ str(x) for x in range(0,25)])

In [58]:
new_test["index"]=range(0,len(new_test))
new_train_1=pd.merge(new_train.reset_index(),genes_train.reset_index()).drop("index",axis=1)
new_test_1=pd.merge(new_test,genes_test.reset_index()).drop("index",axis=1)

C:\Users\rafiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
new_train_1.to_csv("checkpoints_databases/w_working_train_0.csv",index=False,encoding="utf8")
new_test_1.to_csv("checkpoints_databases/w_working_test_0.csv",index=False,encoding="utf8")

In [265]:
new_test_1.shape

(986, 47)